# Table of Contents
 <p><div class="lev1 toc-item"><a href="#VGG19-모델-및-Weight-불러-와서-모델-생성" data-toc-modified-id="VGG19-모델-및-Weight-불러-와서-모델-생성-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>VGG19 모델 및 Weight 불러 와서 모델 생성</a></div><div class="lev2 toc-item"><a href="#Bottleneck-Feature-생성" data-toc-modified-id="Bottleneck-Feature-생성-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Bottleneck Feature 생성</a></div><div class="lev2 toc-item"><a href="#Bottleneck-Feature-저장" data-toc-modified-id="Bottleneck-Feature-저장-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Bottleneck Feature 저장</a></div><div class="lev1 toc-item"><a href="#Transfer-Learning" data-toc-modified-id="Transfer-Learning-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Transfer Learning</a></div><div class="lev2 toc-item"><a href="#FC레이어-변경" data-toc-modified-id="FC레이어-변경-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>FC레이어 변경</a></div><div class="lev2 toc-item"><a href="#모델-다시-생성" data-toc-modified-id="모델-다시-생성-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>모델 다시 생성</a></div><div class="lev2 toc-item"><a href="#모델-다시-학습" data-toc-modified-id="모델-다시-학습-23"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>모델 다시 학습</a></div><div class="lev1 toc-item"><a href="#Fine-Tuning" data-toc-modified-id="Fine-Tuning-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Fine Tuning</a></div>

In [1]:
import tensorflow as tf; print("TensorFLow Version:"+str(tf.__version__))
import keras; print("Keras Version:"+str(keras.__version__))

TensorFLow Version:1.1.0


Using TensorFlow backend.


Keras Version:2.0.3


In [2]:
import os
import h5py
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model 
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [ ]:
from keras import optimizers
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

In [4]:
# path to the model weights file.
#weights_path = 'vgg16_weights.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'dog-cat/train'
validation_data_dir = 'dog-cat/val'
nb_train_samples = 3611 * 2
nb_validation_samples = 444 * 2
nb_epoch = 20

# VGG19 모델 및 Weight 불러 와서 모델 생성

In [5]:
from keras import applications
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [6]:
datagen = ImageDataGenerator(rescale=1./255)

In [10]:
generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=16,
        class_mode=None,
        shuffle=False)


Found 2222 images belonging to 2 classes.


In [11]:
generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=16,
        class_mode=None,
        shuffle=False)

Found 222 images belonging to 2 classes.


## Bottleneck Feature 생성
- model.fit() 학습 결과가 아닌 Predict값을 bottleneck Feature로 사용
- ImageNet의 내용을 학습한 VGG19의 마지막 FC부분을 변경 하지 않았는데도 Predict하는게 의미가 있는가? 

In [8]:
bottleneck_features_train = model.predict_generator(generator, nb_train_samples)
# 24분

In [12]:
bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples)
# 2분

## Bottleneck Feature 저장

In [13]:
np.save(open('bottleneck_features_train.npy', 'wb'), bottleneck_features_train)
np.save(open('bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)

# Transfer Learning

In [14]:
train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
train_labels = np.array([0] * int(nb_train_samples / 2) + [1] * int(nb_train_samples / 2))

validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
validation_labels = np.array([0] * int(nb_validation_samples / 2) + [1] * int(nb_validation_samples / 2))


## FC레이어 변경

In [18]:
model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))



## 모델 다시 생성

In [19]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])



## 모델 다시 학습

In [45]:
# data와 라벨수 강제 맞춤 

x = train_data[1:7223,:,:,:]
y = validation_data[0:888,:,:,:]


model.fit(x, train_labels, #train_data, train_labels,
          nb_epoch=nb_epoch, batch_size=16,
          validation_data=(y, validation_labels)) #validation_data=(validation_data, validation_labels))
#1분 17초

/usr/local/lib/python3.5/dist-packages/keras/models.py:834: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 7222 samples, validate on 888 samples
Epoch 1/20
7222/7222 [==============================] - 6s - loss: 0.7877 - acc: 0.5079 - val_loss: 0.6936 - val_acc: 0.5000
Epoch 2/20
7222/7222 [==============================] - 3s - loss: 0.6942 - acc: 0.5079 - val_loss: 0.6934 - val_acc: 0.5000
Epoch 3/20
7222/7222 [==============================] - 3s - loss: 0.6932 - acc: 0.5141 - val_loss: 0.6941 - val_acc: 0.5000
Epoch 4/20
7222/7222 [==============================] - 3s - loss: 0.6931 - acc: 0.5087 - val_loss: 0.6947 - val_acc: 0.5000
Epoch 5/20
7222/7222 [==============================] - 3s - loss: 0.6922 - acc: 0.5213 - val_loss: 0.6935 - val_acc: 0.5000
Epoch 6/20
7222/7222 [==============================] - 3s - loss: 0.6931 - acc: 0.5152 - val_loss: 0.6936 - val_acc: 0.5000
Epoch 7/20
7222/7222 [==============================] - 3s - loss: 0.6917 - acc: 0.5173 - val_loss: 0.6943 - val_acc: 0.5000
Epoch 8/20
7222/7222 [==============================] - 3s - loss: 0.6918 - ac

In [46]:
model.save_weights(top_model_weights_path)

# Fine Tuning
- 위처럼 마지막 FC만 변경을 하였을때는 Fine Tuning을 하지 않음
- 아래처럼 특정 레이어를 오픈 하였을때는 fine Tuning을 수행 ??
- Fine Tuning을 하려면 위처럼 FC만 변경한 weight값이 있어야 함 ?? OR 그냥 FC변경 안한 FC만 있으면 됨??